## Section 6: Project 1 - Portfolio Valuation With Google Finance

In [1]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# user input
with open("stocks.json") as f:
    stocks = json.load(f)

In [3]:
stocks

{'output_currency': 'USD',
 'stocks': [{'ticker': 'BNS', 'exchange': 'TSE', 'quantity': 100},
  {'ticker': 'GOOGL', 'exchange': 'NASDAQ', 'quantity': 30},
  {'ticker': 'SHOP', 'exchange': 'TSE', 'quantity': 10},
  {'ticker': 'MSFT', 'exchange': 'NASDAQ', 'quantity': 2}]}

In [4]:
def convert_currency(price, input_currency, output_currency):
    if input_currency == output_currency:
        return price
    url = f"https://www.google.com/finance/quote/{input_currency}-{output_currency}"
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")
    conversion_rate = float(
        soup.find("div", attrs={"data-entity-type": "3"})["data-last-price"]
    )
    converted_price = round(price * conversion_rate, 2)
    return converted_price

In [5]:
def get_price(ticker, exchange, output_currency):
    print(f"Crawling {ticker}:{exchange}")
    url = f"https://www.google.com/finance/quote/{ticker}:{exchange}"
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")
    stock_info = soup.find("div", attrs={"data-entity-type": "0"})
    input_currency = stock_info["data-currency-code"]
    price = float(stock_info["data-last-price"])
    converted_price = convert_currency(price, input_currency, output_currency)
    print(f"{input_currency} {price} ==> {output_currency} {converted_price}\n")
    return converted_price

In [6]:
output_currency = stocks["output_currency"]
total_portfolio_value = 0
for stock in stocks["stocks"]:
    price = get_price(stock["ticker"], stock["exchange"], output_currency)
    market_value = price * stock["quantity"]
    stock.update({"price": price, "market_value": market_value})
    total_portfolio_value += market_value
total_portfolio_value = round(total_portfolio_value, 2)

Crawling BNS:TSE
CAD 66.65 ==> USD 49.29

Crawling GOOGL:NASDAQ
USD 123.39 ==> USD 123.39

Crawling SHOP:TSE
CAD 81.8 ==> USD 60.49

Crawling MSFT:NASDAQ
USD 316.95 ==> USD 316.95



In [7]:
# add percent_allocation
for stock in stocks["stocks"]:
    percent_allocation = round((stock["market_value"] / total_portfolio_value) * 100, 2)
    stock.update({"percent_allocation": percent_allocation})

In [8]:
stocks

{'output_currency': 'USD',
 'stocks': [{'ticker': 'BNS',
   'exchange': 'TSE',
   'quantity': 100,
   'price': 49.29,
   'market_value': 4929.0,
   'percent_allocation': 49.94},
  {'ticker': 'GOOGL',
   'exchange': 'NASDAQ',
   'quantity': 30,
   'price': 123.39,
   'market_value': 3701.7,
   'percent_allocation': 37.51},
  {'ticker': 'SHOP',
   'exchange': 'TSE',
   'quantity': 10,
   'price': 60.49,
   'market_value': 604.9,
   'percent_allocation': 6.13},
  {'ticker': 'MSFT',
   'exchange': 'NASDAQ',
   'quantity': 2,
   'price': 316.95,
   'market_value': 633.9,
   'percent_allocation': 6.42}]}

In [9]:
print(pd.DataFrame(stocks["stocks"]).to_markdown(index=False))

| ticker   | exchange   |   quantity |   price |   market_value |   percent_allocation |
|:---------|:-----------|-----------:|--------:|---------------:|---------------------:|
| BNS      | TSE        |        100 |   49.29 |         4929   |                49.94 |
| GOOGL    | NASDAQ     |         30 |  123.39 |         3701.7 |                37.51 |
| SHOP     | TSE        |         10 |   60.49 |          604.9 |                 6.13 |
| MSFT     | NASDAQ     |          2 |  316.95 |          633.9 |                 6.42 |


In [10]:
print(f"Total portfolio value: {output_currency} {total_portfolio_value}")

Total portfolio value: USD 9869.5
